In [2]:
from copy import deepcopy
import cv2
import matplotlib
import numpy as np
import os  # ,fnmatch
import pandas as pd
from pathlib import Path

from brainbox.io.one import SessionLoader
from ibldsp.smooth import smooth_interpolate_savgol  # Ines commented out; need to update iblenv to make this work
from one.api import ONE

# one = ONE()
one = ONE(base_url='https://alyx.internationalbrainlab.org')

/home/ines/miniconda3/envs/iblenv/lib/python3.9/site-packages/one/api.py:1644: UserWarning: Newer cache tables require ONE version 2.10 or greater
  warnings.warn(f'Newer cache tables require ONE version {min_version} or greater')


In [3]:

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx


def load_pose(eid, cam, smoothed=False, manual=True, use_lp=True):

    '''
    cam in left, right, body 
    '''

    if manual:
        pth = one.eid2path(eid)
        if use_lp:
            d = pd.read_parquet(pth / 'alf' / f'_ibl_{cam}Camera.lightningPose.pqt')
        else:
            d = pd.read_parquet(pth / 'alf' / f'_ibl_{cam}Camera.dlc.pqt')
        d['times'] = np.load(one.eid2path(eid) / 'alf'
                    / f'_ibl_{cam}Camera.times.npy')
                    
        ls = [len(d[x]) for x in d]
        if not all(ls == np.mean(ls)):
            lsd = {x:len(d[x]) for x in d}
            print(f'length mismatch: {lsd}')
            print(eid, cam)
            print('cutting times')
            d['times'] = d['times'][:ls[0]]            

    else:
        # load DLC
        sess_loader = SessionLoader(one, eid)
        sess_loader.load_pose(views=[cam])
        d = sess_loader.pose[f'{cam}Camera']
    
    if smoothed:
        print('smoothing dlc traces')
        window = 13 if cam == 'right' else 7
        sers = [x for x in d.keys() if (x[-1] in ['x','y'])]# and 'paw' in x
        for ser in sers:
            d[ser] = smooth_interpolate_savgol(
                d[ser].to_numpy(),
                window=window,order=3, interp_kind='linear')   

    return d




def Viewer(eid, video_type, frame_start, frame_stop, save_video=True, 
           eye_zoom=False, lp=False, masked=True, paws_only=False,
           smooth_dlc = False
          ):
           
    '''
    eid: session id, e.g. '3663d82b-f197-4e8b-b299-7b803a155b84'
    video_type: one of 'left', 'right', 'body'
    save_video: video is saved this local folder

    Example usage to view and save labeled video with wheel angle:
    Viewer('3663d82b-f197-4e8b-b299-7b803a155b84', 'left', [5,7])

    '''

    save_vids_here = Path.home()/'Output LP'


    alf_path = one.eid2path(eid)

    # Download a single video
    video_path = (alf_path / 
        f'raw_video_data/_iblrig_{video_type}Camera.raw.mp4')
    
    if not os.path.isfile(video_path):
        print('mp4 not found locally, downloading it ...')
        video_path = one.load_dataset(eid,
            f'raw_video_data/_iblrig_{video_type}Camera.raw.mp4',
            download_only=True)


    # Download DLC traces and stamps
    Times = one.load_dataset(eid,f'alf/_ibl_{video_type}Camera.times.npy')

    print('loading pose')
    cam = load_pose(eid, video_type, smoothed=smooth_dlc, use_lp=lp)
                                                      
    # get video info
    cap = cv2.VideoCapture(video_path.as_uri())
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    size = (int(cap.get(3)), int(cap.get(4)))

    print(eid,
          ', ',
          video_type,
          ', fsp:',
          fps,
          ', #frames:',
          length,
          ', #stamps:',
          len(Times),
          ', #frames - #stamps = ',
          length - len(Times))

    # pick trial range for which to display stuff
    trials = one.load_object(eid, 'trials')

    print('frame start stop', frame_start, frame_stop)

    '''
    load wheel
    '''

    wheel = one.load_object(eid, 'wheel')
    import brainbox.behavior.wheel as wh
    try:
        pos, t = wh.interpolate_position(
            wheel['timestamps'], wheel['position'], freq=1000)
    except BaseException:
        pos, t = wh.interpolate_position(
            wheel['times'], wheel['position'], freq=1000)

    w_start = find_nearest(t, Times[frame_start])
    w_stop = find_nearest(t, Times[frame_stop])

    # confine to interval
    pos_int = pos[w_start:w_stop]
    t_int = t[w_start:w_stop]

    # alignment of cam stamps and interpolated wheel stamps
    wheel_pos = []
    kk = 0
    for wt in Times[frame_start:frame_stop]:
        wheel_pos.append(pos_int[find_nearest(t_int, wt)])
        kk += 1
        if kk % 3000 == 0:
            print('iteration', kk)

    '''
    DLC related stuff
    '''
    Times = Times[frame_start:frame_stop]
    Frames = np.arange(frame_start, frame_stop)
    
    
    # liklihood threshold
    l_thr = 0.9 if masked else -1
    # l_thr = 0 if masked else -1  # Ines added

    points = [x[:-2] for x in cam.keys() if x[-1] == 'x']

    if video_type != 'body':
        try:
            d = list(points)
            d.remove('tube_top')
            d.remove('tube_bottom')
            points = d
        except:
            pass

    if paws_only:
        p2 = deepcopy(points)
        for point in p2:
            if not 'paw' in point:
                points.remove(point)            

    points = np.array(points)        
        
    # Set values to nan if likelyhood is too low # for pqt: .to_numpy()
    XYs = {}
    zscores = {}  # Ines added
    for point in points:
        x = np.ma.masked_where(
            cam[point + '_likelihood'] < l_thr, cam[point + '_x'])
        x = x.filled(np.nan)
        y = np.ma.masked_where(
            cam[point + '_likelihood'] < l_thr, cam[point + '_y'])
        y = y.filled(np.nan)
        XYs[point] = np.array(
            [x[frame_start:frame_stop], y[frame_start:frame_stop]])    
        # if 'pupil' in point or 'paw' in point:    
        #     zsc = np.ma.masked_where(
        #         cam[point + '_likelihood'] < l_thr, cam[point + '_zscore'])
        #     zsc = zsc.filled(np.nan)
        #     zscores[point] = np.array(
        #         zsc[frame_start:frame_stop])
                   
        

    # Just for 3D testing
    # return XYs

    # Zoom at eye
    if eye_zoom:
        pivot = np.nanmean(XYs['pupil_top_r'], axis=1)
        x0 = int(pivot[0]) - 33
        x1 = int(pivot[0]) + 33
        y0 = int(pivot[1]) - 28
        y1 = int(pivot[1]) + 38
        size = (66, 66)
        dot_s = 1  # [px] for painting DLC dots

    else:
        x0 = 0
        x1 = size[0]
        y0 = 0
        y1 = size[1]
        if video_type == 'left':
            dot_s = 10  # [px] for painting DLC dots
        else:
            dot_s = 5

    if save_video:
    
        loc = (save_vids_here / 
        f'{eid}_{video_type}_frames_{frame_start}_{frame_stop}'
        f'_lp_{lp}.mp4')

        out = cv2.VideoWriter(str(loc),
                              cv2.VideoWriter_fourcc(*'mp4v'),
                              fps,
                              size)  # put , 0 if grey scale

    # writing stuff on frames
    font = cv2.FONT_HERSHEY_SIMPLEX

    if video_type == 'left':
        bottomLeftCornerOfText = (20, 1000)
        fontScale = 4
    else:
        bottomLeftCornerOfText = (10, 500)
        fontScale = 2

    lineType = 2

    # assign a color to each DLC point (now: all points red)
    cmap = matplotlib.cm.get_cmap('Set1')
    CR = np.arange(len(points)) / len(points)

    block = np.ones((2 * dot_s, 2 * dot_s, 3))

    # set start frame
    cap.set(1, frame_start)

    k = 0
    while(cap.isOpened()):
        ret, frame = cap.read()
        gray = frame

        # print wheel angle
        fontColor = (255, 255, 255)
        Angle = round(wheel_pos[k], 2)
        Time = round(Times[k], 3)
        cv2.putText(gray,
                    'Wheel angle: ' + str(Angle),
                    bottomLeftCornerOfText,
                    font,
                    fontScale / 2,
                    fontColor,
                    lineType)

        a, b = bottomLeftCornerOfText
        bottomLeftCornerOfText0 = (int(a * 10 + b / 2), b)
        cv2.putText(gray,
                    '  time: ' + str(Time),
                    bottomLeftCornerOfText0,
                    font,
                    fontScale / 2,
                    fontColor,
                    lineType)
                    
                    
        bottomLeftCornerOfText1 = (a, b - 3* a)
        cv2.putText(gray,
                    'Frame: ' + str(Frames[k]),
                    bottomLeftCornerOfText1,
                    font,
                    fontScale / 2,
                    fontColor,
                    lineType)                    
                    
                    

        # print DLC dots
        ll = 0
        for point in points:

            # Put point color legend
            fontColor = (np.array([cmap(CR[ll])]) * 255)[0][:3]
            a, b = bottomLeftCornerOfText
            if video_type == 'right':
                bottomLeftCornerOfText2 = (a, a * 2 * (1 + ll))
            else:
                bottomLeftCornerOfText2 = (b, a * 2 * (1 + ll))
            fontScale2 = fontScale / 4
            cv2.putText(gray, point,
                        bottomLeftCornerOfText2,
                        font,
                        fontScale2,
                        fontColor,
                        lineType)              

            X0 = XYs[point][0][k]
            Y0 = XYs[point][1][k]
            # if 'pupil' in point or 'paw' in point:
            #     ZSCORE = zscores[point][k]  # Ines added
            # transform for opencv?
            X = Y0
            Y = X0

            if not np.isnan(X) and not np.isnan(Y):
                col = (np.array([cmap(CR[ll])]) * 255)[0][:3]
                # col = np.array([0, 0, 255]) # all points red
                # if 'pupil' in point or 'paw' in point:
                #     # Ines added
                #     if ZSCORE > .5:
                #         col = np.array([0, 0, 255]) # red
                #     elif ZSCORE < .1:
                #         col = np.array([0, 255, 0]) # green
                #     elif .1 <= ZSCORE <= .5:
                #         col = np.array([0, 255, 255]) # yellow
                X = X.astype(int)
                Y = Y.astype(int)
                gray[X - dot_s:X + dot_s, Y - 
                     dot_s:Y + dot_s] = block * col            

            ll += 1

        gray = gray[y0:y1, x0:x1]
        if save_video:
            out.write(gray)
        k += 1
        if k == (frame_stop - frame_start) - 1:
            break

    if save_video:
        out.release()
    cap.release()
    file_path
    print(eid, video_type, frame_start, frame_stop)


In [30]:
import pickle 

ext_qc_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'
os.chdir(ext_qc_path)
ext_qc = pickle.load(open(ext_qc_path + "dlc_left_extended_qc", "rb"))
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb"))
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb"))

use_filter = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'CRITICAL', np.nan])) &
                            (lick_custom['ratio']>5) &
                            (lick_custom['miss_lick_count']<100)]
eids = use_filter['eid'].unique()
files = os.listdir(Path.home()/'Output LP')
for e, eid in enumerate(eids[62:]):
    file_path =  str(eid)+'_left_frames_5000_7000_lp_False.mp4'
    
    if file_path not in files:
        #try:
        Viewer(eid, 'left', 5000, 7000, lp=False)
        # Delete video
        video_file = str(one.eid2path(eid)) + '/raw_video_data/_iblrig_leftCamera.raw.mp4'
        try: 
            os.remove(video_file)
            print(f"File '{video_file}' deleted successfully.")
        except:
            print(f"Could not delete '{video_file}' file.")
        # except:
        #     print('Failed to plot ' + str(eid))

mp4 not found locally, downloading it ...
Downloading: /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_024/2020-06-08/001/raw_video_data/_iblrig_leftCamera.raw.d57b10ad-2b64-49a8-a4dc-39004adf55ae.mp4 Bytes: 10286579343


100%|██████████| 9810.046523094177/9810.046523094177 [26:32<00:00,  6.16it/s]


loading pose
8207abc6-6b23-4762-92b4-82e05bed5143 ,  left , fsp: 60.0 , #frames: 444776 , #stamps: 444776 , #frames - #stamps =  0
frame start stop 5000 7000


/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_024/2020-06-08/001/alf/_ibl_wheel.timestamps.npy: 100%|██████████| 5.39M/5.39M [00:03<00:00, 1.78MB/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_024/2020-06-08/001/alf/_ibl_wheel.position.npy: 100%|██████████| 5.39M/5.39M [00:02<00:00, 1.89MB/s]
/tmp/ipykernel_54901/190198525.py:237: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = matplotlib.cm.get_cmap('Set1')


8207abc6-6b23-4762-92b4-82e05bed5143 left 5000 7000
File '/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_024/2020-06-08/001/raw_video_data/_iblrig_leftCamera.raw.mp4' deleted successfully.
mp4 not found locally, downloading it ...
Downloading: /home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_025/2020-07-26/001/raw_video_data/_iblrig_leftCamera.raw.a9008393-5229-4485-9ac5-67edf0aa82fe.mp4 Bytes: 3440720104


100%|██████████| 3281.3263931274414/3281.3263931274414 [09:16<00:00,  5.90it/s]
/home/ines/Downloads/ONE/alyx.internationalbrainlab.org/zadorlab/Subjects/CSH_ZAD_025/2020-07-26/001/alf/_ibl_leftCamera.times.npy: 100%|██████████| 2.23M/2.23M [00:01<00:00, 1.31MB/s]


loading pose


ValueError: Length of values (278361) does not match length of index (278412)

In [28]:
import pickle 

ext_qc_path = '/home/ines/repositories/representation_learning_variability/Video and wheel/Video QC/'
os.chdir(ext_qc_path)
ext_qc = pickle.load(open(ext_qc_path + "dlc_left_extended_qc", "rb"))
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb"))
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb"))

lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                (lick_custom['videoLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                (lick_custom['ratio']>5) &
                (lick_custom['_videoLeft_pin_state']=='PASS') &
                (lick_custom['_videoLeft_pin_state'].apply(lambda x: x[0] if isinstance(x, list) else False)) &
                (lick_custom['_videoLeft_framerate']=='PASS') &
                (lick_custom['_videoLeft_framerate'].apply(lambda x: x[0] if isinstance(x, list) else False)) &
                (lick_custom['_videoLeft_timestamps'].isin([True, 'PASS'])) &
                (lick_custom['_videoLeft_camera_times']=='PASS') &
                (lick_custom['_videoLeft_camera_times'].apply(lambda x: x[0] if isinstance(x, list) else False)) &
                (lick_custom['_videoLeft_dropped_frames']=='PASS') &
                (lick_custom['_videoLeft_dropped_frames'].apply(lambda x: x[0] if isinstance(x, list) else False)) &
                (lick_custom['_videoLeft_wheel_alignment']=='PASS') &
                (lick_custom['_videoLeft_wheel_alignment'].apply(lambda x: x[0] if isinstance(x, list) else False))       
                            ]

,task,dlcBody,dlcLeft,behavior,dlcRight,videoBody,videoLeft,videoRight,_task_iti_delays,_videoBody_focus,...,eid,_experimenter_task,_videoLeft_video_present,_videoRight_video_present,videoRight`,_videoRight`_pin_state,_videoRight`_wheel_alignment,experimenter,ratio,miss_lick_count


In [35]:
(lick_custom['_videoLeft_wheel_alignment'].apply(lambda x: x[0] if isinstance(x, list) else False))

0    False
0    False
0    False
0    False
0    False
     ...  
0    False
0    False
0    False
0    False
0    False
Name: _videoLeft_wheel_alignment, Length: 308, dtype: bool

In [ ]:
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb"))
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb"))

use_filter = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                             (lick_custom['videoLeft'].isin(['FAIL', 'WARNING', 'PASS', np.nan])) &
                             (lick_custom['ratio']>5) &
                             (lick_custom['_videoLeft_pin_state'][0]=='PASS')
                            ]
eids = use_filter['eid'].unique()
files = os.listdir(Path.home()/'Output LP')

videoLeft_qc = ['eid', 'videoLeft', '_videoLeft_position',  '_videoLeft_pin_state',
                '_videoLeft_framerate', '_videoLeft_resolution',
                '_videoLeft_brightness', '_videoLeft_timestamps',
                '_videoLeft_file_headers', '_videoLeft_camera_times',
                '_videoLeft_focus', '_videoLeft_dropped_frames',
                '_videoLeft_wheel_alignment' ]
dlcLeft_qc = ['eid', 'dlcLeft', '_dlcLeft_time_trace_length_match', '_dlcLeft_trace_all_nan',
              '_dlcLeft_mean_in_bbox', '_dlcLeft_pupil_blocked', 
              '_dlcLeft_lick_detection'
]


for e, eid in enumerate(eids):
    print(eid)
    for k in videoLeft_qc:
        a = use_filter.loc[use_filter['eid']==eid, k]
        print(k + str(list(a)))

In [60]:
use_qc = ext_qc.loc[ext_qc['task'].isin(['PASS', 'WARNING'])]

psth_df = pickle.load(open(ext_qc_path + "dlc_left_extended_psth", "rb"))
lick_custom = pickle.load(open(ext_qc_path + "dlc_left_extended_lick", "rb"))

use_filter = lick_custom.loc[(lick_custom['dlcLeft'].isin(['FAIL', 'CRITICAL', np.nan])) &
                             (lick_custom['videoLeft'].isin(['FAIL', 'CRITICAL', np.nan])) &
                            (lick_custom['ratio']>5) &
                            (lick_custom['miss_lick_count']<100)]
eids = use_filter['eid'].unique()
files = os.listdir(Path.home()/'Output LP')

videoLeft_qc = ['eid', 'videoLeft', '_videoLeft_position',  '_videoLeft_pin_state',
                '_videoLeft_framerate', '_videoLeft_resolution',
                '_videoLeft_brightness', '_videoLeft_timestamps',
                '_videoLeft_file_headers', '_videoLeft_camera_times',
                '_videoLeft_focus', '_videoLeft_dropped_frames',
                '_videoLeft_wheel_alignment' ]
dlcLeft_qc = ['eid', 'dlcLeft', '_dlcLeft_time_trace_length_match', '_dlcLeft_trace_all_nan',
              '_dlcLeft_mean_in_bbox', '_dlcLeft_pupil_blocked', 
              '_dlcLeft_lick_detection'
]

check_eids = ['4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b',
'9a629642-3a9c-42ed-b70a-532db0e86199',
'8c2f7f4d-7346-42a4-a715-4d37a5208535',
'8db36de1-8f17-4446-b527-b5d91909b45a',
'1eac875c-feaa-4a30-b148-059b954b11d8',
'41431f53-69fd-4e3b-80ce-ea62e03bf9c7',
'a66f1593-dafd-4982-9b66-f9554b6c86b5',
'a82800ce-f4e3-4464-9b80-4c3d6fade333',
'6a601cc5-7b79-4c75-b0e8-552246532f82',
'20c112a1-8a42-44e0-a4cd-e5b932f7bda9']
for e, eid in enumerate(eids):
    print(eid)
    for k in videoLeft_qc:
        a = use_filter.loc[use_filter['eid']==eid, k]
        print(k + str(list(a)))

d57df551-6dcb-4242-9c72-b806cff5613a
eid['d57df551-6dcb-4242-9c72-b806cff5613a']
videoLeft['FAIL']
_videoLeft_position[False]
_videoLeft_pin_state['PASS']
_videoLeft_framerate[[True, 60.976]]
_videoLeft_resolution[True]
_videoLeft_brightness[True]
_videoLeft_timestamps[True]
_videoLeft_file_headers[True]
_videoLeft_camera_times[[True, 0]]
_videoLeft_focus[True]
_videoLeft_dropped_frames[[True, 0, 0]]
_videoLeft_wheel_alignment['PASS']
e2b845a1-e313-4a08-bc61-a5f662ed295e
eid['e2b845a1-e313-4a08-bc61-a5f662ed295e']
videoLeft['FAIL']
_videoLeft_position[True]
_videoLeft_pin_state['PASS']
_videoLeft_framerate[[True, 60.981]]
_videoLeft_resolution[True]
_videoLeft_brightness[False]
_videoLeft_timestamps[True]
_videoLeft_file_headers[True]
_videoLeft_camera_times[[True, 0]]
_videoLeft_focus[True]
_videoLeft_dropped_frames[[True, 0, 0]]
_videoLeft_wheel_alignment['PASS']
c8d46ee6-eb68-4535-8756-7c9aa32f10e4
eid['c8d46ee6-eb68-4535-8756-7c9aa32f10e4']
videoLeft['FAIL']
_videoLeft_position[Tru

In [56]:
no_dlc_plot = ['4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b',
'9a629642-3a9c-42ed-b70a-532db0e86199',
'8c2f7f4d-7346-42a4-a715-4d37a5208535',
'8db36de1-8f17-4446-b527-b5d91909b45a',
'1eac875c-feaa-4a30-b148-059b954b11d8',
'41431f53-69fd-4e3b-80ce-ea62e03bf9c7',
'a66f1593-dafd-4982-9b66-f9554b6c86b5',
'a82800ce-f4e3-4464-9b80-4c3d6fade333',
'6a601cc5-7b79-4c75-b0e8-552246532f82',
'20c112a1-8a42-44e0-a4cd-e5b932f7bda9']
no_whisker = ['4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b',
              'cf43dbb1-6992-40ec-a5f9-e8e838d0f643',
'4a45c8ba-db6f-4f11-9403-56e06a33dfa4',
'10fac7a1-919d-4ca7-83c4-4675bd8d9416',
'493170a6-fd94-4ee4-884f-cc018c17eeb9',
'edd22318-216c-44ff-bc24-49ce8be78374',
'73918ae1-e4fd-4c18-b132-00cb555b1ad2',
'25d1920e-a2af-4b6c-9f2e-fc6c65576544',
'eebacd5a-7dcd-4ba6-9dff-ec2a4d2f19e0',
'a4a74102-2af5-45dc-9e41-ef7f5aed88be',
'493170a6-fd94-4ee4-884f-cc018c17eeb9'] 


In [57]:
np.intersect1d(no_dlc_plot, no_whisker)

array(['4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b'], dtype='<U36')

In [ ]:
'4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b' # no dlc ; QC fails for time_trace_length_match - lick plot worked
'9a629642-3a9c-42ed-b70a-532db0e86199' # length does not match; QC fails for time_trace_length_match 
'8c2f7f4d-7346-42a4-a715-4d37a5208535' # length does not match; QC fails for time_trace_length_match 
'8db36de1-8f17-4446-b527-b5d91909b45a' # could not broadcast input array from shape (20,20,3) into shape (0,20,3); QC fails for time_trace_length_match + mean not in box
'1eac875c-feaa-4a30-b148-059b954b11d8' # could not broadcast input array from shape (20,20,3) into shape (0,20,3); QC fails for time_trace_length_match + mean not in box
'41431f53-69fd-4e3b-80ce-ea62e03bf9c7' # could not broadcast inp; QC fails for time_trace_length_match + mean not in box
'a66f1593-dafd-4982-9b66-f9554b6c86b5' # could not broadcast input array from shape (20,20,3) into shape (0,20,3); QC fails for time_trace_length_match + mean not in box
'a82800ce-f4e3-4464-9b80-4c3d6fade333' # length does not match; QC fails for time_trace_length_match + mean not in box
'6a601cc5-7b79-4c75-b0e8-552246532f82' # length does not match; QC fails for time_trace_length_match + mean not in box
'20c112a1-8a42-44e0-a4cd-e5b932f7bda9' # length does not match; QC fails for trace all nan